In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def print_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total_params:          {total_params}\n"
          f"Trainable_params:      {trainable_params}\n"
          f"Non-trainable_params:  {total_params-trainable_params}")

device = torch.device('cuda:1')

train_df = pd.read_json("data/train.jsonl", lines=True)
train_df = pd.DataFrame(train_df, columns=['text', 'label'])
dev_df = pd.read_json("data/dev.jsonl", lines=True)
dev_df = pd.DataFrame(dev_df, columns=['text', 'label'])

In [3]:
class DataframeDataset(Dataset):

    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        row = self.dataframe.iloc[idx]
        return {'text': row.text, 'label': row.label}

In [4]:
def test_model(model, dataframe):
    model.eval()
    preds = []
    targets = []
    loss = 0
    loop = tqdm(DataLoader(DataframeDataset(dataframe), batch_size=32))
    for batch in loop:
        output = tokenizer(batch['text'], padding=True, return_tensors='pt').to(device)
        input_ids = output['input_ids'].to(device)
        attention_mask = output['attention_mask'].to(device)
        labels = batch['label'].to(device)
        result = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss += result.loss.item()
        logits = result.logits
        preds.extend(torch.argmax(logits,axis=1).cpu().tolist())
        targets.extend(labels.cpu().tolist())
    
    loss /= len(loop)
    print("Mean loss: ", loss)
    print("Accuracy: ", accuracy_score(targets, preds))
    print(classification_report(targets, preds))
    return accuracy_score(targets, preds)

In [5]:
model_name = 'Hate-speech-CNERG/dehatebert-mono-english'
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True)
model = model.to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# trainable_params = ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']

# for param_name, param in model.named_parameters():
#     if param_name in trainable_params:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False
        
print_parameters(model)

Total_params:          167357954
Trainable_params:      167357954
Non-trainable_params:  0


In [8]:
optimizer = transformers.AdamW(model.parameters(), lr=1e-6)

max_accuracy = test_model(model, dev_df)
max_state = {key: value.detach().clone() for key, value in model.state_dict().items()}

for epoch in range(25):
    total_loss = 0
    loop = tqdm(DataLoader(DataframeDataset(train_df), shuffle=True, batch_size=64))
    model = model.train()
    for i, batch in enumerate(loop):
        output = tokenizer(batch['text'], padding=True, return_tensors='pt')
        input_ids = output['input_ids'].to(device)
        attention_mask = output['attention_mask'].to(device)
        labels = batch['label'].to(device)
        output = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = output.loss
        total_loss += loss.item()
        loop.set_postfix({'loss': total_loss / (i+1)})
        loss.backward()
        optimizer.step()
    
    print("Epoch: ", epoch+1)
    accuracy = test_model(model, dev_df)
    if accuracy > max_accuracy:
        max_accuracy = accuracy
        max_state = {key: value.detach().clone() for key, value in model.state_dict().items()}

  0%|          | 0/133 [00:00<?, ?it/s, loss=0.401]

Mean loss:  0.9192260634154081
Accuracy:  0.548
              precision    recall  f1-score   support

           0       0.53      0.89      0.66       250
           1       0.65      0.21      0.32       250

    accuracy                           0.55       500
   macro avg       0.59      0.55      0.49       500
weighted avg       0.59      0.55      0.49       500



 19%|█▉        | 3/16 [00:00<00:00, 21.14it/s]

Epoch:  1


  0%|          | 0/133 [00:00<?, ?it/s, loss=0.501]

...

Mean loss:  0.9514499381184578
Accuracy:  0.558
              precision    recall  f1-score   support

           0       0.55      0.70      0.61       250
           1       0.58      0.42      0.48       250

    accuracy                           0.56       500
   macro avg       0.56      0.56      0.55       500
weighted avg       0.56      0.56      0.55       500



 19%|█▉        | 3/16 [00:00<00:00, 20.79it/s]

Epoch:  25


100%|██████████| 16/16 [00:00<00:00, 22.66it/s]

Mean loss:  1.0706781912595034
Accuracy:  0.536
              precision    recall  f1-score   support

           0       0.52      0.90      0.66       250
           1       0.63      0.18      0.28       250

    accuracy                           0.54       500
   macro avg       0.57      0.54      0.47       500
weighted avg       0.57      0.54      0.47       500



In [9]:
model.load_state_dict(max_state)

<All keys matched successfully>

In [13]:
test_model(model, dev_df)

100%|██████████| 16/16 [00:00<00:00, 23.15it/s]

Mean loss:  0.8273804374039173
Accuracy:  0.596
              precision    recall  f1-score   support

           0       0.59      0.62      0.61       250
           1       0.60      0.57      0.59       250

    accuracy                           0.60       500
   macro avg       0.60      0.60      0.60       500
weighted avg       0.60      0.60      0.60       500



0.596

In [14]:
model.save_pretrained("text_model_best")